The goal is to predict user preferences for chatbot responses in a competition using large language models.

In [11]:
import keras_nlp
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns   
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
import plotly.express as px
import os 
os.environ["KERAS_BACKEND"] = "jax"
from tensorflow import keras

# Configuration


In [12]:
class CFG:
    seed = 42 #Random seed to produce similar results in each run
    preset = "deberta_v3_extra_small_en" #Name of the pretrained model
    sequence_length = 512 #Input sequence length
    epochs = 3 #Number of epochs
    batch_sz = 24 #Batch size
    scheduler = 'cosine' #learning rate scheduler
    label2name = {0:'winner_model_a', 1:'winner_model_b', 2:'winner_tie'}
    name2label = {v:k for k,v in label2name.items()}
    class_labels = list(label2name.keys())
    class_names = list(label2name.values())


In [13]:
#Random seed to produce similar results in each run
keras.utils.set_random_seed(CFG.seed)

In this notebook, we will use mixed precision instead of float32 precision for training and inference to reduce GPU memory usage. This will ultimately allow us to use larger batch sizes, thus reducing our training and inference time.

In [14]:
keras.mixed_precision.set_global_policy('mixed_float16')

Your GPU may run slowly with dtype policy mixed_float16 because it does not have compute capability of at least 7.0. Your GPU:
  METAL, no compute capability (probably not an Nvidia GPU)
See https://developer.nvidia.com/cuda-gpus for a list of GPUs and their compute capabilities.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


# Dataset Path



In [15]:
BASE_PATH = '/Users/amirshahcheraghian/Desktop/Desktop/PHD ETS/Python Codes/Kaggle /LMSYS - Chatbot Arena Project'

# Train Data (Import and Cleaning)


In [25]:
df = pd.read_csv(BASE_PATH + '/train.csv')

#remove the square backets
df["prompt"] = df.prompt.map(lambda x:eval(x)[0])

#Remove "null" and replace it with an empty string
df["response_a"] = df.response_a.map(lambda x:eval(x.replace("null","''"))[0]) #Remove "null" and replace it with an empty string
df["response_b"] = df.response_b.map(lambda x:eval(x.replace("null","''"))[0]) #Remove "null" and replace it with an empty string

#For each row, idxmax(axis=1) returns the column name (i.e., the model name or "tie") that has the highest score among the three columns.
df["class_name"] = df[["winner_model_a", "winner_model_b", "winner_tie"]].idxmax(axis=1)

#Use map as an encoder for convert the class names to class labels
df["class_label"] = df["class_name"].map(CFG.name2label)

df.head()

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie,class_name,class_label
0,30192,gpt-4-1106-preview,gpt-4-0613,Is it morally right to try to have a certain p...,The question of whether it is morally right to...,"As an AI, I don't have personal beliefs or opi...",1,0,0,winner_model_a,0
1,53567,koala-13b,gpt-4-0613,What is the difference between marriage licens...,A marriage license is a legal document that al...,A marriage license and a marriage certificate ...,0,1,0,winner_model_b,1
2,65089,gpt-3.5-turbo-0613,mistral-medium,explain function calling. how would you call a...,Function calling is the process of invoking or...,Function calling is the process of invoking a ...,0,0,1,winner_tie,2
3,96401,llama-2-13b-chat,mistral-7b-instruct,How can I create a test set for a very rare ca...,Creating a test set for a very rare category c...,When building a classifier for a very rare cat...,1,0,0,winner_model_a,0
4,198779,koala-13b,gpt-3.5-turbo-0314,What is the best way to travel from Tel-Aviv t...,The best way to travel from Tel Aviv to Jerusa...,The best way to travel from Tel-Aviv to Jerusa...,0,1,0,winner_model_b,1


# Test Data (import and cleaning)

In [27]:
#Load the test data
test_df = pd.read_csv(BASE_PATH + '/test.csv')

#Remove the square brackets and get the first element of the list
test_df["prompt"] = test_df.prompt.map(lambda x:eval(x)[0])

#Remove "null" and replace it with an empty string
test_df["response_a"] = test_df.response_a.map(lambda x:eval(x.replace("null","''"))[0])
test_df["response_b"] = test_df.response_b.map(lambda x:eval(x.replace("null","''"))[0])

test_df.head()

,id,prompt,response_a,response_b
0,136060,"I have three oranges today, I ate an orange ye...",You have two oranges today.,You still have three oranges. Eating an orange...
1,211333,You are a mediator in a heated political debat...,Thank you for sharing the details of the situa...,Mr Reddy and Ms Blue both have valid points in...
2,1233961,How to initialize the classification head when...,When you want to initialize the classification...,To initialize the classification head when per...
